In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Load data
data = fetch_california_housing()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=42)

In [2]:
def create_mc_dropout_model(input_shape):
    inputs = keras.Input(shape=input_shape)
    x = layers.Dense(64, activation='relu')(inputs)
    x = layers.Dropout(0.5)(x, training=True)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.5)(x, training=True)
    outputs = layers.Dense(1)(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='mse')
    return model

def mc_dropout_predict(model, X, n_iter=100):
    f = tf.function(lambda x: model(x, training=True))
    preds = np.array([f(X).numpy().flatten() for _ in range(n_iter)])
    return np.mean(preds, axis=0), np.var(preds, axis=0)

mc_dropout_model = create_mc_dropout_model((X_train.shape[1],))
mc_dropout_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1, verbose=2)
mc_mean, mc_var = mc_dropout_predict(mc_dropout_model, X_test, n_iter=100)

Epoch 1/20
465/465 - 3s - 6ms/step - loss: 13415.1426 - val_loss: 23.7247
Epoch 2/20
465/465 - 1s - 2ms/step - loss: 935.2645 - val_loss: 4.7440
Epoch 3/20
465/465 - 1s - 2ms/step - loss: 259.4278 - val_loss: 5.4560
Epoch 4/20
465/465 - 1s - 2ms/step - loss: 105.3934 - val_loss: 3.4117
Epoch 5/20
465/465 - 1s - 2ms/step - loss: 48.1893 - val_loss: 2.8969
Epoch 6/20
465/465 - 1s - 2ms/step - loss: 44.3223 - val_loss: 2.9917
Epoch 7/20
465/465 - 1s - 2ms/step - loss: 23.2927 - val_loss: 3.4571
Epoch 8/20
465/465 - 1s - 2ms/step - loss: 15.9502 - val_loss: 2.7901
Epoch 9/20
465/465 - 1s - 2ms/step - loss: 22.7107 - val_loss: 2.4936
Epoch 10/20
465/465 - 1s - 2ms/step - loss: 10.7581 - val_loss: 2.0387
Epoch 11/20
465/465 - 1s - 2ms/step - loss: 7.9373 - val_loss: 2.0170
Epoch 12/20
465/465 - 1s - 2ms/step - loss: 8.5125 - val_loss: 1.7045
Epoch 13/20
465/465 - 1s - 2ms/step - loss: 6.1865 - val_loss: 1.7869
Epoch 14/20
465/465 - 1s - 2ms/step - loss: 6.0497 - val_loss: 1.5164
Epoch 15/20


In [4]:
import tensorflow_probability as tfp

def create_mfvi_model(input_shape):
    inputs = keras.Input(shape=(input_shape,))
    x = tfp.layers.DenseReparameterization(64, activation='relu')(inputs)
    x = tfp.layers.DenseReparameterization(64, activation='relu')(x)
    outputs = tfp.layers.DenseReparameterization(1)(x)
    model = keras.Model(inputs, outputs)
    negloglik = lambda y, rv_y: -rv_y.log_prob(y)
    model.compile(optimizer='adam', loss=negloglik)
    return model

def mfvi_predict(model, X, n_iter=100):
    preds = np.array([model(X).mean().numpy().flatten() for _ in range(n_iter)])
    return np.mean(preds, axis=0), np.var(preds, axis=0)

mfvi_model = create_mfvi_model(X_train.shape[1])
mfvi_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1, verbose=2)
mfvi_mean, mfvi_var = mfvi_predict(mfvi_model, X_test, n_iter=100)

AttributeError: 'tuple' object has no attribute 'rank'